In [5]:
import json
import tensorflow as tf
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import warnings
from keras.preprocessing.text import tokenizer_from_json

warnings.filterwarnings("ignore")
warnings.simplefilter(action="ignore", category=FutureWarning)

from dataset import *
from lstm import *

dataset = load_tokenized_sentences("../datasets/words/bajki-raw.pickle")

with open("../datasets/words/books-bajki-raw-tokenizer_100000.json", "r") as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)

print(
    {
        "size": len(dataset),
        "tokenizer": tokenizer.num_words,
        "str": dataset[1][:500],
        "tokenizer": tokenizer.num_words,
    }
)

{'size': 41, 'tokenizer': 100000, 'str': 'była raz mała słodka dzieweczka, którą kochał każdy, kto ją tylko ujrzał, a najwięcej kochała ją babcia  nie wiedziała wprost, co jej dać. pewnego razu podarowała jej kapturek z czerwonego aksamitu, a dziewczynce tak się ten kapturek podobał, że nie chciała nosić żadnego innego, toteż nazwano ją czerwonym kapturkiem.\npewnego razu rzekła matka do czerwonego kapturka:\n oto masz, dziecko, w koszyku placek i flaszkę wina, zanieś to babci, która jest chora i słaba, i ucieszy się bardzo tym podarunkie'}


In [6]:
class_weights = calculate_class_weights(tokenizer, alpha=0.4) 
SKIP = 2
total_words = tokenizer.num_words
batch_size = 256

padding: Literal["pre", "post"] = "pre"
dataset_size = calculate_dataset_size(dataset, tokenizer, LEN_MAX_LIMIT, SKIP)
gen = dataset_generator(
    dataset, tokenizer, LEN_MIN_LIMIT, LEN_MAX_LIMIT, SKIP, batch_size, padding=padding
)
a, b = next(gen)
print(a.shape, b.shape)
print(a[0], b[0])
print('class_weights', len(class_weights), class_weights[[1,2,3,4,5,-5,-4,-3,-2,-1]])

(256, 50) (256,)
[    0     0    59     5   368    25  3199 72391    10   248 18540 26661
  7902    17     8   198  2224  4293     1 53560    51  2319    40   220
    38    62    11  3220   831    10    72 23949   166   500  5410     1
 49554   397 41638 62449     8   341 12676     3 78716   101 22002     2
    65  1382] 1
class_weights 100001 [ 1.          1.10900746  1.14756841  1.22807962  1.26507838 73.39486412
 73.39486412 73.39486412 73.39486412 73.39486412]


In [7]:
from keras.models import load_model

model: Model = load_model('../lstm_models/model_best_5.h5', custom_objects={"WeightedSCCE":  losses.SparseCategoricalCrossentropy})
model.compile(optimizer='adam', loss=WeightedSCCE(class_weights))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            5000000   
                                                                 
 bidirectional (Bidirectiona  (None, 50, 1024)         2306048   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 50, 1024)         6295552   
 nal)                                                            
                                                                 
 dropout (Dropout)           (None, 50, 1024)          0         
                                                                 
 bidirectional_2 (Bidirectio  (None, 1024)             6295552   
 nal)                                                            
                                                        

In [8]:
model.fit(
    gen,
    steps_per_epoch=dataset_size // batch_size,
    epochs=20,
    verbose=1,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            "../lstm_models/model_best_5_retrain.h5",
            monitor="loss",
            save_best_only=True,
            save_weights_only=False,
        )
    ]
    + [
        PredictCallback(
            "dawno temu czerwony kapturek poszedł do lasu. w rękach trzymała koszyczek",
            60,
            LEN_MAX_LIMIT,
            tokenizer,
            padding,
            _t,
        )
        for _t in [0, 0.1, 0.2, 0.3, 0.5, 0.8, 1.0]
    ],
)

Start epoch 0 of training; Temperature: 0.0 Generated text: dawno temu czerwony kapturek poszedł do lasu. w rękach trzymała koszyczek z i z obnażonymi w nim górę i w jednej chwili setka metrów pod powierzchnią morza. nautilus płynął w pobliżu placu. tam nie było w tym czasie w tym miejscu ani jednego z nich ani śladu w tym miejscu nie było w tym nic z tego nie było w tym czasie w tym sierpniu w tym celu w r. ,
Start epoch 0 of training; Temperature: 0.1 Generated text: dawno temu czerwony kapturek poszedł do lasu. w rękach trzymała koszyczek z i z obnażonymi w nim górę i w jednej chwili setka metrów pod powierzchnią morza. nautilus wypłynął na powierzchnię morza na powierzchnię morza, nie było w tym czasie regularnych drzew i w których się podniósł, się w ciągu jednej z czapeczki i zachód, z rozpuszczonymi tysiącami świec z których parły się i w powietrzu się w górę i w
Start epoch 0 of training; Temperature: 0.2 Generated text: dawno temu czerwony kapturek poszedł do lasu. w rękach tr

In [ ]:
from tqdm import tqdm
        
text_seed = "dawno temu czerwony kapturek poszedł do lasu"
for temperature in tqdm([0, 0.1, 0.2, 0.3, 0.5, 0.8, 1.0]):
    pr = PredictCallback(text_seed, 250, LEN_MAX_LIMIT, tokenizer, padding, temperature, model)
    text = pr.generate_text()
    with open(f'../generated_texts/text_temperature_{temperature:.2f}', 'w') as f:
        f.write(f'Seed: {text_seed}\n')
        f.write(text)